In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl

In [ ]:
train_dataset = pkl.load(open('/content/drive/MyDrive/mediaeval2016/Twitter/myTemp/train_dataset.pkl', 'rb'))
test_dataset=pkl.load(open('/content/drive/MyDrive/mediaeval2016/Twitter/myTemp/test_dataset.pkl','rb'))

In [ ]:
train_dataset[0]#0:fake,1:Real

['need feds solve #bostonbombing #4chan',
 'need feds solve #bostonbombing #4chan',
 'boston_fake_03.jpg',
 0]

In [ ]:
def get_data(dataset):
  text=[]
  target=[]
  for data in dataset:
    text.append(data[1])
    target.append(data[3])
  return text,target
train_text,train_target=get_data(train_dataset)
test_text,test_target=get_data(test_dataset)
len(train_text),len(test_text)

(10740, 932)

In [ ]:
train_text[:5]

['need feds solve #bostonbombing #4chan',
 'pic comparison #boston suspect sunil tripathi fbi released images video missing poster decide',
 'completely convinced sunil tripathi fellow',
 '4chan bombing throwing',
 '4chan thinks pictures bomber']

In [ ]:
train_df=pd.DataFrame(list(zip(train_text,train_target)),columns=['text','label'])
test_df=pd.DataFrame(list(zip(test_text,test_target)),columns=['text','label'])
train_df.head()

,text,label
0,need feds solve #bostonbombing #4chan,0
1,pic comparison #boston suspect sunil tripathi ...,0
2,completely convinced sunil tripathi fellow,0
3,4chan bombing throwing,0
4,4chan thinks pictures bomber,0


In [ ]:
print(train_df['label'].value_counts())
print(test_df['label'].value_counts())

0    5839
1    4901
Name: label, dtype: int64
0    518
1    414
Name: label, dtype: int64


## Text Preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_text(text):
    text = re.sub(r"<U\+2019>|<U\+2018>", "\'", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " had ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"n\'t", " not", text)
    text= re.sub(r"<U\+.+>",' ',text)
    text = re.sub(r"[\!\?\.\,\+\-\$\%\^\>\<\=\:\;\*\(\)\{\}\[\]\/\~\&\|]", " ", text)
    return text


In [ ]:
words=[]
def create_corpus(messages):
  corpus=[]
  for i in range(len(messages)):
    review=clean_text(messages[i])
    review.lower()
    review=review.split()
    review=[word for word in review if word.isalpha()]
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    for word in review:
      if word not in words:
        words.append(word)
    review=' '.join(review).strip()
    if len(review)==0:
      review='no text'
    corpus.append(review)
  return corpus 


In [ ]:
len(train_text)

10740

In [ ]:
corpus_train=create_corpus(train_text)
corpus_test=create_corpus(test_text)
len(words)

4808

In [ ]:
len(corpus_train),len(corpus_test)

(10740, 932)

In [ ]:
corpus_train[:5]

['need fed solv',
 'pic comparison suspect sunil tripathi fbi releas imag video miss poster decid',
 'complet convinc sunil tripathi fellow',
 'bomb throw',
 'think pictur bomber']

##One Hot Encoding

In [ ]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [ ]:
voc_size=6000
onehot_repr_train=[one_hot(words,voc_size) for words in corpus_train]
onehot_repr_test=[one_hot(words,voc_size) for words in corpus_test]
len(onehot_repr_train),len(onehot_repr_test)

(10740, 932)

In [ ]:
onehot_repr_test

[[865, 455, 4331, 2482, 3026, 5028, 4654],
 [4707, 4461, 2949, 4066, 1921, 5366, 4901, 5223, 1169, 2445],
 [4707, 2177, 4901],
 [4707, 1751, 4407, 525, 5975, 3156, 4407, 525, 376],
 [2365, 5723],
 [4707, 3108, 4901, 2445, 5223],
 [5842, 4901, 2445],
 [4707, 5842, 4901, 2445],
 [4707, 5842, 5366, 648, 1622, 3681, 3643, 5126, 3552, 5217, 727, 1642],
 [811, 1622, 2498, 3108, 4415, 2498, 1699, 1262, 2321],
 [5217, 140, 4901, 1622, 3681, 5919, 3858, 5455],
 [4707, 3234, 2445, 1622, 5028, 3108, 4901],
 [4707, 5539, 3234, 1622, 1602],
 [4707, 1622, 2498, 3108, 4415, 2498, 1699, 1262, 2321, 1417],
 [1751, 4407, 525, 5975, 3156, 4407, 525, 376],
 [4707, 1751, 4407, 525, 5975, 3156, 4407, 525, 376],
 [1983],
 [4707, 2445, 1169],
 [4707, 4699, 2375, 2445, 1622, 4901, 2752, 3056, 3550],
 [4707, 4699, 2375, 2445, 1622, 4901, 2752, 3056, 3550],
 [4707, 4747, 2557],
 [2752, 3056, 3550, 4926],
 [4747, 2557],
 [4707, 4461, 2949, 4066, 1921, 5366, 4901, 5223, 1169, 2445],
 [4707, 4461, 2949, 4066, 1921,

In [ ]:
onehot_repr_train[0],corpus_train[0]

([4705, 182, 5391], 'need fed solv')

In [ ]:
len(onehot_repr_train+onehot_repr_test)

11672

In [ ]:
max_sent_len=0
for sent in onehot_repr_train+onehot_repr_test:
  l=len(sent)
  if max_sent_len<l:
    max_sent_len=l
max_sent_len

17

In [ ]:
embedded_docs_train=pad_sequences(onehot_repr_train,padding='pre',maxlen=max_sent_len)
embedded_docs_test=pad_sequences(onehot_repr_test,padding='pre',maxlen=max_sent_len)
embedded_docs_train

array([[   0,    0,    0, ..., 4705,  182, 5391],
       [   0,    0,    0, ..., 1784, 1963, 5597],
       [   0,    0,    0, ...,  351, 4188, 1335],
       ...,
       [   0,    0,    0, ..., 4122, 1907, 4901],
       [   0,    0,    0, ..., 1166, 4551, 2893],
       [   0,    0,    0, ..., 2375, 4434, 1907]], dtype=int32)

In [ ]:
embedded_docs_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 4705,  182, 5391], dtype=int32)

#Model

In [ ]:
from keras.layers import Embedding,LSTM,Bidirectional,Dense,Dropout,GlobalMaxPool1D
from keras.models import Sequential
import keras

In [ ]:
test_dataset

[['tristesse #bataclan sold musiciens amp public en osmose quelques minutes avant le drame #paixauxvictimes',
  'sadness #bataclan sold musicians & public in osmosis a few minutes before the drama #peaceauxvictimes',
  'attacks_paris_1.jpg',
  0],
 ['rt \\@proyecto40 #últimahora espectacular fotografía sacada antes los ataques en la sala del teatro bataclan',
  'rt \\ @ project40 # last hour spectacular photograph taken before the attacks in the theater room bataclan',
  'attacks_paris_1.jpg',
  0],
 ['rt \\@javivi1976 #bataclan esta noche antes los atentados \\@playradiovlc',
  'rt \\ @ javivi1976 #bataclan tonight before the attacks \\ @playradiovlc',
  'attacks_paris_1.jpg',
  0],
 ['rt \\@pizzigatas el hombre tiene que establecer final para la guerra sino ésta establecerá fin para la humanidad #paris',
  'rt \\ @pizzigatas man has to establish an end for war otherwise it will establish an end for humanity #paris',
  'attacks_paris_1.jpg',
  0],
 ['No Text', 'No Text', 'attacks_pari

In [ ]:
embedded_dim=40#dimension of embedding layer
model=Sequential()
model.add(Embedding(voc_size,embedded_dim,input_length=max_sent_len))
model.add(Bidirectional(LSTM(80)))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 17, 40)            240000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 160)               77440     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 161       
Total params: 317,601
Trainable params: 317,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint_cb=keras.callbacks.ModelCheckpoint("/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5",save_best_only=True,monitor='val_accuracy',mode='auto',verbose=1)

In [ ]:
model.fit(embedded_docs_train,np.array(train_target),validation_data=(embedded_docs_test,np.array(test_target)),epochs=50,batch_size=128,callbacks=[checkpoint_cb])

Epoch 1/50
84/84 [==============================] - 4s 17ms/step - loss: 0.5856 - accuracy: 0.6523 - val_loss: 0.8146 - val_accuracy: 0.5976

Epoch 00001: val_accuracy improved from -inf to 0.59764, saving model to /content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5
Epoch 2/50
84/84 [==============================] - 1s 9ms/step - loss: 0.2685 - accuracy: 0.8820 - val_loss: 1.0060 - val_accuracy: 0.6094

Epoch 00002: val_accuracy improved from 0.59764 to 0.60944, saving model to /content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5
Epoch 3/50
84/84 [==============================] - 1s 10ms/step - loss: 0.2284 - accuracy: 0.9054 - val_loss: 0.9940 - val_accuracy: 0.6062

Epoch 00003: val_accuracy did not improve from 0.60944
Epoch 4/50
84/84 [==============================] - 1s 9ms/step - loss: 0.2069 - accuracy: 0.9133 - val_loss: 1.3166 - val_accuracy: 0.6127

Epoch 00004: val_accuracy improved from 0.60944 to

In [ ]:
model=keras.models.load_model('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5')

In [ ]:
y_pred_test=model.predict_classes(embedded_docs_test)
y_pred_test

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
    

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

In [ ]:
accuracy=accuracy_score(test_target,y_pred_test)
accuracy

0.6126609442060086

In [ ]:
print(classification_report(test_target,y_pred_test))

              precision    recall  f1-score   support

           0       0.65      0.67      0.66       518
           1       0.57      0.55      0.56       414

    accuracy                           0.61       932
   macro avg       0.61      0.61      0.61       932
weighted avg       0.61      0.61      0.61       932

